<a href="https://colab.research.google.com/github/imeunu/Capstone_PBL/blob/main/Speaker_verification/endtoend_SV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np

In [ ]:
X_data= np.load('/content/drive/MyDrive/Speaker_verification/data/e2e_train_X.npy')
Y_data= np.load('/content/drive/MyDrive/Speaker_verification/data/e2e_train_Y.npy')

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y= train_test_split(X_data, Y_data, test_size= 0.2,
                                                   stratify= Y_data)

In [ ]:
train_X= np.array(train_X)
test_X= np.array(test_X)
train_Y= np.array(train_Y)
test_Y= np.array(test_Y)

In [ ]:
len(np.unique(train_Y))

100

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.models import Sequential, load_model, model_from_json 
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dense,Activation,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,Dropout,AveragePooling2D,BatchNormalization,Softmax
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.metrics import accuracy_score
from tensorflow.python.client import device_lib

In [ ]:
#Model
inp_shape = train_X.shape[1:]
def generate_model_cnn(mode='words', pooling=0, layers=3, kernel=11):
    model = Sequential()
    print("=================================CNN=====================================")
    model.add(Conv2D(filters=128, kernel_size=(3,3), strides = 1, padding ='same', input_shape=inp_shape, activation = 'relu')) #add convolution layer
    model.add(AveragePooling2D(pool_size=(3,3), strides=2)) #pooling layer
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides = 1, padding ='same', activation = 'relu'))
    model.add(AveragePooling2D(pool_size=(3,3), strides=None))
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides = 1, padding ='same', activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.1))
    model.add(Dense(len(np.unique(train_Y)), activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.1))
    model.add(Dense(len(np.unique(train_Y)), activation = 'linear'))
    model.add(Softmax())
    return model

def generate_model(mode='words', summary=True):
    model = generate_model_cnn(mode=mode, pooling=0)
    if summary:
        model.summary()
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

callbacks_list  = [
    EarlyStopping( monitor = 'loss',  min_delta=0.001, patience=100,
                  verbose=1, mode='auto'),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=7, verbose=1, min_delta=1e-4)]

model = generate_model(mode='words',summary=True)
history = model.fit(train_X, train_Y, epochs=500, batch_size=100, validation_data=(test_X,test_Y),callbacks=callbacks_list)

=================================CNN=====================================
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 14, 44, 128)       1280      
_________________________________________________________________
average_pooling2d_24 (Averag (None, 6, 21, 128)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 6, 21, 64)         73792     
_________________________________________________________________
average_pooling2d_25 (Averag (None, 2, 7, 64)          0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 2, 7, 64)          36928     
_________________________________________________________________
flatten_12 (Flatten)         (None, 896)               0         
_____________________________________________

In [ ]:
import keras
new_model= keras.models.Sequential(model.layers[:-1])

## 1명의 화자 등록했을 경우

In [ ]:
enroll_X= np.load('/content/drive/MyDrive/Speaker_verification/data/1_enroll/1_enroll_X.npy')
enroll_Y= np.load('/content/drive/MyDrive/Speaker_verification/data/1_enroll/1_enroll_Y.npy')
eval_X=np.load('/content/drive/MyDrive/Speaker_verification/data/1_enroll/1_eval_X.npy')
eval_Y=np.load('/content/drive/MyDrive/Speaker_verification/data/1_enroll/1_eval_Y.npy')

In [ ]:
enroll_X= np.array(enroll_X)
enroll_X= np.expand_dims(enroll_X, axis= -1)
eval_X= np.array(eval_X)
eval_X= np.expand_dims(eval_X, axis= -1)
mean_vector= np.mean(model.predict(enroll_X), axis=0)

In [ ]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
encoded_eval_Y=[]
for eval_speaker in eval_Y:
  if eval_speaker== enroll_Y[0]:
    encoded_eval_Y.append(0)
  else:
    encoded_eval_Y.append(1) 

In [ ]:
predict_eval= model.predict(eval_X)

In [ ]:
cos_sim_list=[]
for i in predict_eval:
  cos_sim_list.append(cos_sim(mean_vector, i))

In [ ]:
real_pred=[]
for i in cos_sim_list:
  if i>=0.3:
    real_pred.append(0)
  else:
    real_pred.append(1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(encoded_eval_Y, real_pred)

array([[ 6,  4],
       [ 2, 48]])

In [ ]:
accuracy_score(encoded_eval_Y, real_pred)

0.9

## 2명 등록했을 경우

In [ ]:
enroll_X= np.load('/content/drive/MyDrive/Speaker_verification/data/2_enroll/2_enroll_X.npy')
enroll_Y= np.load('/content/drive/MyDrive/Speaker_verification/data/2_enroll/2_enroll_Y.npy')
eval_X=np.load('/content/drive/MyDrive/Speaker_verification/data/2_enroll/2_eval_X.npy')
eval_Y=np.load('/content/drive/MyDrive/Speaker_verification/data/2_enroll/2_eval_Y.npy')

In [ ]:
enroll_X= np.array(enroll_X)
enroll_X= np.expand_dims(enroll_X, axis= -1)
eval_X= np.array(eval_X)
eval_X= np.expand_dims(eval_X, axis= -1)
enroll_Y= np.array(enroll_Y)
eval_Y= np.array(eval_Y)

In [ ]:
encoded_eval_Y=[]
for i in eval_Y:
  if i== np.unique(enroll_Y)[0]:
    encoded_eval_Y.append(0)
  elif i== np.unique(enroll_Y)[1]:
    encoded_eval_Y.append(1)
  else:
    encoded_eval_Y.append(2)

In [ ]:
predict_enroll= model.predict(enroll_X)
predict_eval= model.predict(eval_X)

In [ ]:
for ind in range(len(np.unique(enroll_Y))):
  globals()['class_X_'+str(ind)]=[]

In [ ]:
for ind, speaker in enumerate(enroll_Y):
  if speaker== np.unique(enroll_Y)[0]:
    class_X_0.append(predict_enroll[ind])
  else:
    class_X_1.append(predict_enroll[ind])

In [ ]:
for i in range(len(np.unique(enroll_Y))):
  globals()['mean_vector_'+str(i)]= np.mean(globals()['class_X_'+str(i)], axis=0)

In [ ]:
for ind in range(len(np.unique(enroll_Y))):
  globals()['cos_sim_list_'+str(ind)]= np.array([])
  for i in predict_eval:
    tmp= cos_sim(i, globals()['mean_vector_'+str(ind)])
    globals()['cos_sim_list_'+str(ind)]=np.append(globals()['cos_sim_list_'+str(ind)], tmp)
  globals()['cos_sim_list_'+str(ind)]= globals()['cos_sim_list_'+str(ind)].reshape(-1,1)

In [ ]:
real_pred=[]
for ind in range(len(eval_X)):
  max_value= max(cos_sim_list_0[ind], cos_sim_list_1[ind], 0.3)
  if max_value==cos_sim_list_0[ind]:
    real_pred.append(0)
  elif max_value==cos_sim_list_1[ind]:
    real_pred.append(1)
  else:
    real_pred.append(2)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(encoded_eval_Y, real_pred)

array([[ 6,  1,  3],
       [ 2,  7,  1],
       [ 2,  4, 44]])

In [ ]:
accuracy_score(encoded_eval_Y, real_pred)

0.8142857142857143

## 3명을 등록했을때

In [ ]:
enroll_X= np.load('/content/drive/MyDrive/Speaker_verification/data/3_enroll/3_enroll_X.npy')
enroll_Y= np.load('/content/drive/MyDrive/Speaker_verification/data/3_enroll/3_enroll_Y.npy')
eval_X=np.load('/content/drive/MyDrive/Speaker_verification/data/3_enroll/3_eval_X.npy')
eval_Y=np.load('/content/drive/MyDrive/Speaker_verification/data/3_enroll/3_eval_Y.npy')

In [ ]:
enroll_X= np.array(enroll_X)
enroll_X= np.expand_dims(enroll_X, axis= -1)
eval_X= np.array(eval_X)
eval_X= np.expand_dims(eval_X, axis= -1)
enroll_Y= np.array(enroll_Y)
eval_Y= np.array(eval_Y)

In [ ]:
encoded_eval_Y=[]
for i in eval_Y:
  if i== np.unique(enroll_Y)[0]:
    encoded_eval_Y.append(0)
  elif i== np.unique(enroll_Y)[1]:
    encoded_eval_Y.append(1)
  elif i== np.unique(enroll_Y)[2]:
    encoded_eval_Y.append(2)    
  else:
    encoded_eval_Y.append(3)

In [ ]:
predict_enroll= model.predict(enroll_X)
predict_eval= model.predict(eval_X)

In [ ]:
for ind in range(len(np.unique(enroll_Y))):
  globals()['class_X_'+str(ind)]=[]

for ind, speaker in enumerate(enroll_Y):
  if speaker== np.unique(enroll_Y)[0]:
    class_X_0.append(predict_enroll[ind])
  elif speaker== np.unique(enroll_Y)[1]:
    class_X_1.append(predict_enroll[ind])
  else:
    class_X_2.append(predict_enroll[ind])

for i in range(len(np.unique(enroll_Y))):
  globals()['mean_vector_'+str(i)]= np.mean(globals()['class_X_'+str(i)], axis=0)

for ind in range(len(np.unique(enroll_Y))):
  globals()['cos_sim_list_'+str(ind)]= np.array([])
  for i in predict_eval:
    tmp= cos_sim(i, globals()['mean_vector_'+str(ind)])
    globals()['cos_sim_list_'+str(ind)]=np.append(globals()['cos_sim_list_'+str(ind)], tmp)
  globals()['cos_sim_list_'+str(ind)]= globals()['cos_sim_list_'+str(ind)].reshape(-1,1)

In [ ]:
real_pred=[]
for ind in range(len(eval_X)):
  max_value= max(cos_sim_list_0[ind], cos_sim_list_1[ind], cos_sim_list_2[ind], 0.35)
  if max_value==cos_sim_list_0[ind]:
    real_pred.append(0)
  elif max_value==cos_sim_list_1[ind]:
    real_pred.append(1)
  elif max_value==cos_sim_list_2[ind]:
    real_pred.append(2)
  else:
    real_pred.append(3)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(encoded_eval_Y, real_pred)

array([[ 6,  0,  1,  3],
       [ 0,  5,  2,  3],
       [ 2,  0,  6,  2],
       [ 2,  1,  3, 44]])

In [ ]:
accuracy_score(encoded_eval_Y, real_pred)

0.7625

## 4명을 등록

In [ ]:
enroll_X= np.load('/content/drive/MyDrive/Speaker_verification/data/4_enroll/4_enroll_X.npy')
enroll_Y= np.load('/content/drive/MyDrive/Speaker_verification/data/4_enroll/4_enroll_Y.npy')
eval_X=np.load('/content/drive/MyDrive/Speaker_verification/data/4_enroll/4_eval_X.npy')
eval_Y=np.load('/content/drive/MyDrive/Speaker_verification/data/4_enroll/4_eval_Y.npy')

In [ ]:
enroll_X= np.array(enroll_X)
enroll_X= np.expand_dims(enroll_X, axis= -1)
eval_X= np.array(eval_X)
eval_X= np.expand_dims(eval_X, axis= -1)
enroll_Y= np.array(enroll_Y)
eval_Y= np.array(eval_Y)

In [ ]:
encoded_eval_Y=[]
for i in eval_Y:
  if i== np.unique(enroll_Y)[0]:
    encoded_eval_Y.append(0)
  elif i== np.unique(enroll_Y)[1]:
    encoded_eval_Y.append(1)
  elif i== np.unique(enroll_Y)[2]:
    encoded_eval_Y.append(2)    
  elif i== np.unique(enroll_Y)[3]:
    encoded_eval_Y.append(3)    
  else:
    encoded_eval_Y.append(4)

predict_enroll= model.predict(enroll_X)
predict_eval= model.predict(eval_X)

In [ ]:
for ind in range(len(np.unique(enroll_Y))):
  globals()['class_X_'+str(ind)]=[]

for ind, speaker in enumerate(enroll_Y):
  if speaker== np.unique(enroll_Y)[0]:
    class_X_0.append(predict_enroll[ind])
  elif speaker== np.unique(enroll_Y)[1]:
    class_X_1.append(predict_enroll[ind])
  elif speaker== np.unique(enroll_Y)[2]:
    class_X_2.append(predict_enroll[ind])
  else:
    class_X_3.append(predict_enroll[ind])

for i in range(len(np.unique(enroll_Y))):
  globals()['mean_vector_'+str(i)]= np.mean(globals()['class_X_'+str(i)], axis=0)

for ind in range(len(np.unique(enroll_Y))):
  globals()['cos_sim_list_'+str(ind)]= np.array([])
  for i in predict_eval:
    tmp= cos_sim(i, globals()['mean_vector_'+str(ind)])
    globals()['cos_sim_list_'+str(ind)]=np.append(globals()['cos_sim_list_'+str(ind)], tmp)
  globals()['cos_sim_list_'+str(ind)]= globals()['cos_sim_list_'+str(ind)].reshape(-1,1)

In [ ]:
real_pred=[]
for ind in range(len(eval_X)):
  max_value= max(cos_sim_list_0[ind], cos_sim_list_1[ind], cos_sim_list_2[ind],cos_sim_list_3[ind], 0.35)
  if max_value==cos_sim_list_0[ind]:
    real_pred.append(0)
  elif max_value==cos_sim_list_1[ind]:
    real_pred.append(1)
  elif max_value==cos_sim_list_2[ind]:
    real_pred.append(2)
  elif max_value==cos_sim_list_3[ind]:
    real_pred.append(3)  
  else:
    real_pred.append(4)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(encoded_eval_Y, real_pred)

array([[ 7,  0,  1,  0,  2],
       [ 0,  6,  0,  1,  3],
       [ 0,  0,  5,  2,  3],
       [ 0,  2,  0,  6,  2],
       [ 6,  2,  1,  3, 38]])

In [ ]:
accuracy_score(encoded_eval_Y, real_pred)

0.6888888888888889